This time, we are going to work with an already defined $Ca^{2+}$ imaging pipeline. Start as always by importing `datajoint`.

In [ ]:
import datajoint as dj

And now import the `calium.py` module with the pipeline definition.

In [ ]:
import calcium as ca

The module defines a pipeline as can be seen with `Di`.

In [ ]:
dj.Di(ca)

In [ ]:
ca.SegmentationMethod()

We can see existing scans.

In [ ]:
ca.Scan()

...and fetch the scans data!

In [ ]:
a = ca.Scan.fetch()

Fetched data is returned as `numpy` structured array.

In [ ]:
a['wavelength']

We can find all slices across all scans in `CorrectedScan.Slice()` table.

In [ ]:
ca.CorrectedScan.Slice()

Instead of fetching all data at once, you can fetch only the primary keys by fetching `'KEY'`

In [ ]:
for key in ca.CorrectedScan.Slice.fetch('KEY'):
    print(dict((ca.Mouse() & key).fetch1()))

In [ ]:
keys = ca.CorrectedScan.Slice.fetch('KEY')

and you can use this to restrict and view one slice at a time...

In [ ]:
ca.CorrectedScan.Slice & keys[0]

...and fetch data for only a specific slice. You can also fetch specific attribute only (e.g. 'template') and not the entire row of data:

In [ ]:
m = (ca.CorrectedScan.Slice & keys[0]).fetch1('template')

Let's now take a look at the fetched slice.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import rcParams

In [ ]:
plt.imshow(m)

In [ ]:
slices = ca.CorrectedScan.Slice()
len(slices)

We can also iterate through the slices in the table and plot them side by side. Iterator returns restricted relation.

In [ ]:
slices

In [ ]:
rcParams['figure.figsize'] = (10,16)
from itertools import count
c = count()
for m in slices:
    plt.subplot(3, 2, next(c)+1)
    plt.imshow(m['template'])

In [ ]:
ca.CorrectedScan()

In [ ]:
ca.CorrectedScan.SliceImage()

In [ ]:
import numpy as np
rcParams['figure.figsize'] = (10,16)
for i, key in enumerate(ca.CorrectedScan.Slice.fetch('KEY')):
    im1, im2 = (ca.CorrectedScan.SliceImage & key).fetch('average_image', order_by='channel')
    plt.subplot(3, 2, i+1)
    plt.imshow(np.stack((im1/im1.max(),im2/im2.max(),im2*0),axis=2))

In [ ]:
ca.RF() & 'segmentation_method=2'